In [2]:
# Import numpy and OpenCV
import numpy as np
import cv2
 
# Read input video
video_path = r'F:\rotating_headplate\221216_MJG_DMM000_RotationTest\TSeries-12162022-1035-002.avi'
cap = cv2.VideoCapture(video_path)
 
# Get frame count
n_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) 
 
# Get width and height of video stream
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
 
# Define the codec for output video
fourcc = cv2.VideoWriter_fourcc(*'MJPG')
 
# Set up output video
fps=30
save_path = r'F:\rotating_headplate\221216_MJG_DMM000_RotationTest\cv2_derotation_002.mp4'
out = cv2.VideoWriter(save_path, fourcc, fps, (w, h))

In [9]:
# Read first frame
_, prev = cap.read() 
 
# Convert frame to grayscale
prev_gray = cv2.cvtColor(prev, cv2.COLOR_BGR2GRAY)

# Pre-define transformation-store array
transforms = np.zeros((n_frames-1, 3), np.float32) 
 
for i in range(n_frames-2):
    # Detect feature points in previous frame
    prev_pts = cv2.goodFeaturesToTrack(prev_gray,
                                     maxCorners=200,
                                     qualityLevel=0.01,
                                     minDistance=30,
                                     blockSize=3)

    # Read next frame
    success, curr = cap.read()
    if not success:
        break

    # Convert to grayscale
    curr_gray = cv2.cvtColor(curr, cv2.COLOR_BGR2GRAY) 

    # Calculate optical flow (i.e. track feature points)
    curr_pts, status, err = cv2.calcOpticalFlowPyrLK(prev_gray, curr_gray, prev_pts, None) 

    # Sanity check
    assert prev_pts.shape == curr_pts.shape 

    # Filter only valid points
    idx = np.where(status==1)[0]
    prev_pts = prev_pts[idx]
    curr_pts = curr_pts[idx]

    #Find transformation matrix
    m, _ = cv2.estimateAffine2D(prev_pts, curr_pts) #will only work with OpenCV-3 or less

    # Extract traslation
    dx = m[0,2]
    dy = m[1,2]

    # Extract rotation angle
    da = np.arctan2(m[1,0], m[0,0])

    # Store transformation
    transforms[i] = [dx,dy,da]

    # Move to next frame
    prev_gray = curr_gray

    print("Frame: " + str(i) +  "/" + str(n_frames) + " -  Tracked points : " + str(len(prev_pts)))

Frame: 0/9000 -  Tracked points : 192
Frame: 1/9000 -  Tracked points : 197
Frame: 2/9000 -  Tracked points : 200
Frame: 3/9000 -  Tracked points : 200
Frame: 4/9000 -  Tracked points : 196
Frame: 5/9000 -  Tracked points : 196
Frame: 6/9000 -  Tracked points : 200
Frame: 7/9000 -  Tracked points : 191
Frame: 8/9000 -  Tracked points : 199
Frame: 9/9000 -  Tracked points : 200
Frame: 10/9000 -  Tracked points : 200
Frame: 11/9000 -  Tracked points : 198
Frame: 12/9000 -  Tracked points : 197
Frame: 13/9000 -  Tracked points : 200
Frame: 14/9000 -  Tracked points : 197
Frame: 15/9000 -  Tracked points : 197
Frame: 16/9000 -  Tracked points : 199
Frame: 17/9000 -  Tracked points : 198
Frame: 18/9000 -  Tracked points : 192
Frame: 19/9000 -  Tracked points : 195
Frame: 20/9000 -  Tracked points : 197
Frame: 21/9000 -  Tracked points : 200
Frame: 22/9000 -  Tracked points : 198
Frame: 23/9000 -  Tracked points : 192
Frame: 24/9000 -  Tracked points : 196
Frame: 25/9000 -  Tracked points : 

In [8]:
_

array([[ 9.96015114e-01,  2.15346916e-03, -2.59795178e-01],
       [-5.54374552e-04,  9.98299105e-01, -4.82834758e-02]])

In [ ]:
# Compute trajectory using cumulative sum of transformations
trajectory = np.cumsum(transforms, axis=0)